In [ ]:
import numpy as np
import pickle
import yaml

In [ ]:
def calculate_stats(dirname):
    with open(f'benchmark/{dirname}/inference_time_list.pkl', 'rb') as f:
        inference_time_list =pickle.load(f)
    with open(f'benchmark/{dirname}/sampling_time_list.pkl', 'rb') as f:
        sampling_time_list =pickle.load(f)
    with open(f'benchmark/{dirname}/solver_step_time_list.pkl', 'rb') as f:
        solver_step_time_list =pickle.load(f)
    with open(f'benchmark/{dirname}/num_density_solve_list.pkl', 'rb') as f:
        num_density_solve_list =pickle.load(f)
    
    with open(f'benchmark/{dirname}/optimized.yaml', 'r') as f:
        kernel_stat = yaml.safe_load(f)
    kernel_stat_mean = {}
    for key in kernel_stat:
        for elapsed_mean in kernel_stat[key]:
            if elapsed_mean != -1:
                kernel_stat_mean[key] = elapsed_mean
                break
    
    step_kernel_time_except_solve = kernel_stat_mean["clear_acceleration"]+ kernel_stat_mean["compute_particle_boundary_analytic(BoxDistance)"]+ kernel_stat_mean["update_particle_grid"]+ kernel_stat_mean["make_neighbor_list"]+ kernel_stat_mean["compute_density"]+ kernel_stat_mean["compute_viscosity"]+ kernel_stat_mean["drive_n_ellipse"]+ kernel_stat_mean["calculate_cfl_v2"]+ kernel_stat_mean["advect_and_init_pressure"]+ kernel_stat_mean["calculate_isph_diagonal_adv_density"]+ kernel_stat_mean["compute_pressure_accels"]+ kernel_stat_mean["kinematic_integration"]
    simulation_kernel_sum_list = []
    
    usher_kernel_time = kernel_stat_mean["update_particle_grid"]+ kernel_stat_mean["make_neighbor_list"]+ kernel_stat_mean["compute_density"]+ kernel_stat_mean["make_neighbor_list"]+ kernel_stat_mean["compute_particle_boundary_analytic(BoxDistance)"]+ kernel_stat_mean["compute_sample_vorticity"]+ kernel_stat_mean["sample_position_density"]+ kernel_stat_mean["compute_sample_velocity"]

    
    simulation_python_list = []
    for frame_id, item in enumerate(solver_step_time_list):
        if len(item)>0:
            simulation_python_list.append(np.sum(item))
            
            frame_num_density_solve = np.sum(num_density_solve_list[frame_id])
#             print(len(item))
            simulation_time = kernel_stat_mean['isph_solve_iteration'] * frame_num_density_solve + step_kernel_time_except_solve * len(item)
            simulation_kernel_sum_list.append(simulation_time)
            
    simulation_kernel_avg = np.mean(simulation_kernel_sum_list)
    inference_avg = np.mean(inference_time_list[1:])
    print('simulation kernel', simulation_kernel_avg, np.mean(simulation_python_list[1:]))
    print('usher kernel', usher_kernel_time, 'python', np.mean(sampling_time_list[1:]))
    print('inference', inference_avg)
    return simulation_kernel_avg+usher_kernel_time + 0.4464 #+ inference_avg

In [ ]:
calculate_stats('.alcache-optimize-validate-rl-0.015')

In [ ]:
calculate_stats('.alcache-optimize-validate-rl-0.0125')

In [ ]:
calculate_stats('.alcache-optimize-validate-rl-0.011')

In [ ]:
calculate_stats('.alcache-optimize-validate-rl-0.01')

In [ ]:
calculate_stats('.alcache-optimize-validate-rl-0.009')